In [539]:
import numpy as np
import pandas as pd
import sklearn

In [540]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [541]:
train_x = pd.read_csv("/content/drive/My Drive/train_x.csv")
test_x = pd.read_csv("/content/drive/My Drive/test_x.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (114) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [542]:
all_nan_cols = ["勤務地　最寄駅3（駅名）", "応募先　名称","勤務地　最寄駅3（沿線名）","（派遣先）勤務先写真コメント","勤務地　最寄駅3（分）","無期雇用派遣","未使用.14","（派遣以外）応募後の流れ","（派遣先）概要　従業員数","電話応対なし","週払い","固定残業制 残業代 下限","未使用.11","フリー項目　内容","先輩からのメッセージ","未使用.10","未使用.8","経験必須","固定残業制 残業代に充当する労働時間数 下限","ブロックコード2","未使用.4","未使用.7","メモ","ブロックコード3","WEB面接OK","17時以降出社OK","寮・社宅あり","ネットワーク関連のスキルを活かす","Wワーク・副業可能","固定残業制 残業代に充当する労働時間数 上限","プログラム関連のスキルを活かす","未使用.15","未使用.12","エルダー（50〜）活躍中","人材紹介","主婦(ママ)・主夫歓迎","ブロックコード1","フリー項目　タイトル","未使用.1","ブランクOK","未使用.20","募集形態","勤務地　最寄駅3（駅からの交通手段）","応募先　最寄駅（沿線名）","仕事写真（下）　写真1　ファイル名","未使用.16","仕事写真（下）　写真3　ファイル名","オープニングスタッフ","応募先　所在地　ブロックコード","応募先　所在地　都道府県","応募先　最寄駅（駅名）","外国人活躍中・留学生歓迎","未使用.17","未使用.9","日払い","未使用","未使用.18","未使用.22","未使用.5","勤務地　周辺情報","仕事写真（下）　写真2　ファイル名","バイク・自転車通勤OK","仕事写真（下）　写真2　コメント","未使用.3","未使用.2","WEB関連のスキルを活かす","未使用.6","給与　経験者給与下限","学生歓迎","固定残業制 残業代 上限","未使用.19","給与　経験者給与上限","未使用.21","待遇・福利厚生","シニア（60〜）歓迎","ベンチャー企業","少人数の職場","仕事写真（下）　写真3　コメント","応募先　備考","応募先　所在地　市区町村","仕事写真（下）　写真1　コメント","未使用.13","応募拠点","これまでの採用者例","（派遣先）概要　勤務先名（フリガナ）"]
#全てが欠損値のカラムを削除
train_x = train_x.drop(all_nan_cols, axis=1)
test_x = test_x.drop(all_nan_cols, axis=1)

In [543]:
all_same_data_cols = ["勤務地固定","週1日からOK","ミドル（40〜）活躍中","ルーティンワークがメイン","対象者設定　年齢下限","動画コメント","給与/交通費　給与支払区分","CAD関連のスキルを活かす","固定残業制","公開区分","20代活躍中","検索対象エリア","就業形態区分","30代活躍中","雇用形態","Dip JobsリスティングS","資格取得支援制度あり","対象者設定　年齢上限","社会保険制度あり","動画タイトル","残業月10時間未満","履歴書不要","研修制度あり","DTP関連のスキルを活かす","新卒・第二新卒歓迎","産休育休取得事例あり","動画ファイル名","対象者設定　性別"]
#全て同じデータが格納されているカラムの削除
train_x = train_x.drop(all_same_data_cols, axis=1)
test_x = test_x.drop(all_same_data_cols, axis=1)

In [544]:
train_y = pd.read_csv("/content/drive/My Drive/train_y.csv")
train_y.head()

,お仕事No.,応募数 合計
0,2203566799,1.945910
1,2203546900,3.951244
2,2203546900,3.295837
3,2203545344,3.970292
4,2203545344,3.784190


In [545]:
#行削除の工程で次元の不一致が発生してしまうのを防ぐため、一度目的変数と説明変数を統合する
apply_number = train_y["応募数 合計"]
train_x["応募数 合計"] = apply_number

In [546]:
#統合による混乱を防ぐため、変数名変更
train_data = train_x
train_data["応募数 合計"]

0        1.945910
1        3.951244
2        3.295837
3        3.970292
4        3.784190
           ...   
15848    0.000000
15849    0.000000
15850    0.000000
15851    0.693147
15852    0.000000
Name: 応募数 合計, Length: 15853, dtype: float64

In [547]:
#重複データの削除
train_data = train_data.drop_duplicates()
test_x = test_x.drop_duplicates()

In [548]:
train_data = train_data.reset_index()
test_x = test_x.reset_index()

In [549]:
train_data = train_data.drop(["index"],axis=1)
test_x = test_x.drop(["index"],axis=1)

In [550]:
str_cols = ["勤務地　最寄駅2（駅名）","勤務地　最寄駅2（沿線名）","お仕事名","休日休暇　備考","期間・時間　勤務時間","（派遣先）概要　事業内容","（紹介予定）年収・給与例","（紹介予定）休日休暇","派遣会社のうれしい特典","お仕事のポイント（仕事PR）","（派遣先）職場の雰囲気","勤務地　最寄駅1（駅名）","給与/交通費　備考"]

In [551]:
train_data = train_data.drop(str_cols, axis=1)
test_x = test_x.drop(str_cols, axis=1)

In [552]:
#使い物にならなさそうなカラムの削除
invalid_cols = ["（派遣先）勤務先写真ファイル名","勤務地　最寄駅2（分）",]
train_data = train_data.drop(invalid_cols, axis=1)
test_x = test_x.drop(invalid_cols, axis=1)

In [553]:
#欠損値を平均値により補完
male_percentage = train_data["（派遣先）配属先部署　男女比　男"]
male_percentage = male_percentage.fillna(male_percentage.mean())
train_data["（派遣先）配属先部署　男女比　男"] = male_percentage

male_percentage = test_x["（派遣先）配属先部署　男女比　男"]
male_percentage = male_percentage.fillna(male_percentage.mean())
test_x["（派遣先）配属先部署　男女比　男"] = male_percentage

In [554]:
people_number = train_data["（派遣先）配属先部署　人数"]
people_number = people_number.fillna(people_number.mean())
train_data["（派遣先）配属先部署　人数"] = people_number

people_number = test_x["（派遣先）配属先部署　人数"]
people_number = people_number.fillna(test_x.mean())
test_x["（派遣先）配属先部署　人数"] = test_x

In [555]:
take_minute1 = train_data["勤務地　最寄駅1（分）"]
take_minute1 = take_minute1.fillna(take_minute1.mean())
train_data["勤務地　最寄駅1（分）"] = take_minute1

take_minute1 = train_data["勤務地　最寄駅1（分）"]
take_minute1 = take_minute1.fillna(take_minute1.mean())
train_data["勤務地　最寄駅1（分）"] = take_minute1

In [556]:
age_ave = train_data["（派遣先）配属先部署　平均年齢"]
age_ave = age_ave.fillna(age_ave.mean())
train_data["（派遣先）配属先部署　平均年齢"] = age_ave

age_ave = test_x["（派遣先）配属先部署　平均年齢"]
age_ave = age_ave.fillna(age_ave.mean())
test_x["（派遣先）配属先部署　平均年齢"] = age_ave

In [557]:
train_data['掲載期間　開始日'] = pd.to_datetime(train_data["掲載期間　開始日"])
train_data['期間・時間　勤務開始日'] = pd.to_datetime(train_data["期間・時間　勤務開始日"])
train_data['掲載期間　終了日'] = pd.to_datetime(train_data["掲載期間　終了日"])

test_x['掲載期間　開始日'] = pd.to_datetime(test_x["掲載期間　開始日"])
test_x['期間・時間　勤務開始日'] = pd.to_datetime(test_x["期間・時間　勤務開始日"])
test_x['掲載期間　終了日'] = pd.to_datetime(test_x["掲載期間　終了日"])

In [558]:
date_cols = ["掲載期間　開始日","期間・時間　勤務開始日","掲載期間　終了日"]
train_data = train_data.drop(date_cols,axis=1)
test_x = test_x.drop(date_cols,axis=1)

In [559]:
ohe_cols = ["拠点番号","（紹介予定）入社後の雇用形態","（紹介予定）雇用形態備考","勤務地　最寄駅2（駅からの交通手段）","（紹介予定）入社時期","期間･時間　備考"]

In [560]:
train_data = pd.get_dummies(train_data, drop_first=True,columns=ohe_cols, dummy_na=True, prefix='dummy',prefix_sep='')
test_x = pd.get_dummies(test_x, drop_first=True,columns=ohe_cols, dummy_na=True, prefix='dummy',prefix_sep='')

In [561]:
y = train_data["応募数 合計"]
X = train_data.drop(["お仕事No.","応募数 合計"],axis=1)
X

,職場の様子,休日休暇(月曜日),（派遣先）配属先部署 男女比 男,大手企業,交通費別途支給,（派遣先）配属先部署 人数,残業月20時間以上,職種コード,1日7時間以下勤務OK,短時間勤務OK(1日4h以内),駅から徒歩5分以内,学校・公的機関（官公庁）,土日祝のみ勤務,Wordのスキルを活かす,勤務地 最寄駅1（分）,派遣スタッフ活躍中,大量募集,Accessのスキルを活かす,休日休暇(火曜日),平日休みあり,（派遣先）概要 勤務先名（漢字）,フラグオプション選択,（派遣）応募後の流れ,期間・時間 勤務期間,派遣形態,週2・3日OK,勤務先公開,Excelのスキルを活かす,16時前退社OK,正社員登用あり,残業月20時間未満,勤務地 備考,英語力不要,休日休暇(日曜日),社員食堂あり,10時以降出社OK,英語以外の語学力を活かす,休日休暇(祝日),外資系企業,服装自由,...,dummy603徳島,dummy604高知,dummy701福岡,dummy702北九州,dummy703大分,dummy704佐賀,dummy705長崎,dummy706熊本,dummy707宮崎,dummy708鹿児島,dummy709沖縄,dummynan,dummy1.0,dummy2.0,dummynan,dummyアルバイト社員,dummyパート社員,dummy契約員,dummy契約契約社員,dummy契約社員,dummy正社員,dummynan,dummy2.0,dummy3.0,dummynan,dummy◆1ヶ月後,dummy◆2ヶ月後,dummy◆3ヶ月後,dummy◆4ヶ月後,dummy◆5ヶ月後,dummy◆6ヶ月後,dummynan,dummy◆開始日はご相談可能です。詳しくはお問合せください！,dummy◆開始日はご相談可能です。詳しくはお気軽にお問合せください！,dummy◆開始日はご相談可能です！詳しくはお問い合せください！,dummy◆開始日はご相談可能です！詳しくはお問い合わせください！,dummy◆開始日はご相談可能です！詳しくはお問い合わせ下さい！,dummy◆開始日はご相談可能です！詳しくはお問合せください！,dummy延長可能性：有,dummynan
0,3,0,6.000000,0,0,12.000000,0,22010,1,0,1,1,0,1,3.000000,1,0,0,0,0,NaN,1,＊最後までご覧いただき、ありがとうございます＊<BR><BR>「わたしでもチャレンジできるか...,5,0,0,0,1,0,0,0,東京都足立区,1,1,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,3,0,2.000000,1,0,33.000000,0,20020,1,0,1,0,0,1,1.000000,1,0,0,0,0,NaN,3,＊最後までご覧いただき、ありがとうございます＊<BR><BR>「わたしでもチャレンジできるか...,5,1,0,0,1,0,1,0,東京都千代田区,1,1,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,3,0,2.000000,1,0,33.000000,0,20020,1,0,1,0,0,1,1.000000,1,0,0,0,0,NaN,3,＊最後までご覧いただき、ありがとうございます＊<BR><BR>「わたしでもチャレンジできるか...,5,1,0,0,1,0,1,0,東京都千代田区,1,1,1,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
3,2,0,8.000000,0,0,5.000000,1,20020,0,0,0,1,0,1,6.000000,0,0,0,0,0,NaN,3,＊最後までご覧いただき、ありがとうございます＊<BR><BR>「わたしでもチャレンジできるか...,5,1,0,0,1,0,1,0,東京都中央区,1,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
4,2,0,8.000000,0,0,5.000000,1,20020,0,0,0,1,0,1,6.000000,0,0,0,0,0,NaN,3,＊最後までご覧いただき、ありがとうございます＊<BR><BR>「わたしでもチャレンジできるか...,5,1,0,0,1,0,1,0,東京都中央区,1,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14040,3,0,0.000000,0,0,12.000000,0,20040,1,0,1,0,0,1,6.000000,0,0,0,0,0,NaN,0,＊最後までご覧いただき、ありがとうございます＊<BR><BR>「わたしでもチャレンジできるか...,5,0,0,0,1,1,0,0,静岡県静岡市葵区,1,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
14041,2,0,5.001063,0,0,14.186576,0,20040,1,1,0,0,0,0,15.000000,0,0,0,1,0,NaN,0,＊最後までご覧いただき、ありがとうございます＊<BR><BR>「わたしでもチャレンジできるか...,3,0,1,0,0,1,0,0,静岡県浜松市東区,1,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
14042,3,0,8.000000,1,0,13.000000,0,20020,0,0,0,0,0,1,6.556764,0,0,0,0,1,NaN,0,＊最後までご覧いただき、ありがとうございます＊<BR><BR>「わたしでもチャレンジできるか...,5,0,0,0,1,0,0,0,静岡県浜松市東区,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
14043,3,0,5.000000,0,0,12.000000,1,22030,0,0,1,0,0,1,4.000000,0,0,0,0,0,NaN,0,＊最後までご覧いただき、ありがとうございます＊<BR><BR>「わたしでもチャレンジできるか...,3,0,0,0,1,0,0,0,静岡県静岡市葵区,1,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1


In [562]:
not_important_cols = ["（派遣先）配属先部署","給与/交通費　給与上限","勤務地　最寄駅1（沿線名）","（派遣）応募後の流れ","勤務地　備考","（派遣先）概要　勤務先名（漢字）","（紹介予定）待遇・福利厚生","勤務地　備考","扶養控除内","休日休暇(月曜日)","Excelのスキルを活かす","（派遣先）配属先部署　男女比　女","WEB登録OK","派遣形態","給与/交通費　交通費","土日祝のみ勤務","経験者優遇","PCスキル不要","英語以外の語学力を活かす","紹介予定派遣","応募資格","仕事内容"]
X = X.drop(not_important_cols,axis=1)
test_x = test_x.drop(not_important_cols,axis=1)

In [563]:
X = X.drop(X.columns[np.isnan(X).any()], axis=1)
test_x = test_x.dropna(how='all').dropna(how='all', axis=1)

In [564]:
def fill_missing_columns(df_a, df_b):
    columns_for_b = set(df_a.columns) - set(df_b.columns)
    for column in columns_for_b:
        df_b[column] = 0
    columns_for_a = set(df_b.columns) - set(df_a.columns)
    for column in columns_for_a:
        df_a[column] = 0

fill_missing_columns(X, test_x)

X.sort_index(axis=1, inplace=True)
test_x.sort_index(axis=1, inplace=True)

In [565]:
from sklearn import preprocessing
X = X.values 
min_max_scaler = preprocessing.MinMaxScaler()
X_scaled = min_max_scaler.fit_transform(X)
X = pd.DataFrame(X_scaled)

In [566]:
test_x = test_x.values
tx_scaled = min_max_scaler.fit_transform(test_x)
test_x = pd.DataFrame(tx_scaled)

In [567]:
from sklearn.model_selection import train_test_split

In [568]:
X_train,X_test,y_train,y_test = train_test_split(X, y,test_size=0.4,random_state=0)

In [569]:
import lightgbm as lgb

In [570]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test)

In [571]:
lgbm_params = {
        # 回帰問題
        'objective': 'regression',
        # RMSE (平均二乗誤差平方根) の最小化を目指す
        'metric': 'rmse',
    }

In [572]:
model = lgb.train(lgbm_params, lgb_train, valid_sets=lgb_eval)

[1]	valid_0's rmse: 0.785062
[2]	valid_0's rmse: 0.77236
[3]	valid_0's rmse: 0.761298
[4]	valid_0's rmse: 0.75276
[5]	valid_0's rmse: 0.745514
[6]	valid_0's rmse: 0.738068
[7]	valid_0's rmse: 0.73243
[8]	valid_0's rmse: 0.726993
[9]	valid_0's rmse: 0.722318
[10]	valid_0's rmse: 0.71848
[11]	valid_0's rmse: 0.715337
[12]	valid_0's rmse: 0.711687
[13]	valid_0's rmse: 0.708737
[14]	valid_0's rmse: 0.707001
[15]	valid_0's rmse: 0.704405
[16]	valid_0's rmse: 0.702219
[17]	valid_0's rmse: 0.700643
[18]	valid_0's rmse: 0.699526
[19]	valid_0's rmse: 0.697902
[20]	valid_0's rmse: 0.696974
[21]	valid_0's rmse: 0.696078
[22]	valid_0's rmse: 0.693904
[23]	valid_0's rmse: 0.691901
[24]	valid_0's rmse: 0.690254
[25]	valid_0's rmse: 0.688139
[26]	valid_0's rmse: 0.686575
[27]	valid_0's rmse: 0.685232
[28]	valid_0's rmse: 0.683883
[29]	valid_0's rmse: 0.683077
[30]	valid_0's rmse: 0.682125
[31]	valid_0's rmse: 0.681768
[32]	valid_0's rmse: 0.680713
[33]	valid_0's rmse: 0.679915
[34]	valid_0's rmse: 0.

In [573]:
import pickle

file = 'trained_model.pkl'
pickle.dump(model, open(file, 'wb'))

from google.colab import files
files.download("trained_model.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [574]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred

array([0.7846008 , 0.16207535, 0.76256152, ..., 0.76163271, 0.38067014,
       0.0229405 ])

In [575]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(rmse)

0.6601845856700239


In [576]:
pred_comp = pd.DataFrame({"テスト":y_test,
                         "予測":y_pred})
pred_comp.to_csv("pred_comp2.csv",index=False)
from google.colab import files
files.download("pred_comp2.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [577]:
cols = list(X.columns)         
f_importance = np.array(model.feature_importance()) # 特徴量重要度の算出
f_importance = f_importance / np.sum(f_importance) 
df_importance = pd.DataFrame({'feature':cols, 'importance':f_importance})
df_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート
display(df_importance)

''' cols = list(train_data.drop(["お仕事No.","応募数 合計","勤務地　備考","（紹介予定）待遇・福利厚生","（派遣先）配属先部署","勤務地　最寄駅1（沿線名）","（派遣先）概要　勤務先名（漢字）","扶養控除内","休日休暇(月曜日)","Excelのスキルを活かす","（派遣先）配属先部署　男女比　女","WEB登録OK","正社員登用あり","派遣形態","給与/交通費　交通費","土日祝休み","土日祝のみ勤務","経験者優遇","PCスキル不要","英語以外の語学力を活かす","紹介予定派遣","dummy105山形","dummy212千葉"],axis=1).columns)         
f_importance = np.array(model.feature_importance()) # 特徴量重要度の算出
f_importance = f_importance / np.sum(f_importance) 
''' 
''' print(len(cols))
print(f_importance.shape)
print(cols[00])  '''

'''
df_importance = pd.DataFrame({'feature':cols, 'importance':f_importance})
df_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート
display(df_importance) '''

,feature,importance
122,122,0.117333
102,102,0.095333
131,131,0.076667
124,124,0.074000
132,132,0.066667
...,...,...
68,68,0.000000
28,28,0.000000
66,66,0.000000
64,64,0.000000


"\ndf_importance = pd.DataFrame({'feature':cols, 'importance':f_importance})\ndf_importance = df_importance.sort_values('importance', ascending=False) # 降順ソート\ndisplay(df_importance) "

In [578]:
y_pred = model.predict(test_x, num_iteration=model.best_iteration)
y_pred

array([1.78974834, 2.29283466, 3.29394254, ..., 1.35363073, 0.15956462,
       0.85287962])

In [579]:
test_x = pd.read_csv("/content/drive/My Drive/test_x.csv")
pred_comp = pd.DataFrame({"お仕事No.":test_x["お仕事No."],
    "応募数 合計":y_pred})
pred_comp.to_csv("pred_comp.csv",index=False)
from google.colab import files
files.download("pred_comp.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [580]:
''' df_importance.to_csv("data_importance2.csv")
from google.colab import files
files.download("data_importance2.csv") '''

' df_importance.to_csv("data_importance2.csv")\nfrom google.colab import files\nfiles.download("data_importance2.csv") '